In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
# Read data
df = spark.read.table("retail_catalog.retail_schema.features")

In [0]:
Transformed_df = df.toPandas()
Transformed_df.head()

In [0]:
#ordinal field for segment column
# Define the mapping dictionary
segment_mapping = {
    'Low-Value': 1,
    'Medium-Value': 2,
    'High-Value': 3
}

# Apply the mapping to create a new column with ordinal values
Transformed_df['cust_segment_ordinal'] = Transformed_df['customer_segment'].map(segment_mapping)
Transformed_df.head()

In [0]:
#ordinal field for age column
# Define the bins for the age ranges
bins = [18, 30, 40, 50, np.inf]

# Define the labels for the new ordinal values
labels = [1, 2, 3, 4]

# Use pd.cut() to create the new 'age_ordinal' column
# right=False ensures the ranges are inclusive of the lower bound
# (e.g., [18, 30), [30, 40), etc.)
Transformed_df['age_ordinal'] = pd.cut(Transformed_df['age'], bins=bins, labels=labels, right=False)

In [0]:
Transformed_df['age_ordinal']= Transformed_df['age_ordinal'].astype(int)
Transformed_df.head()

In [0]:
Transformed_df['invoice_date'] = pd.to_datetime(Transformed_df['invoice_date'])

In [0]:
Transformed_df.dtypes

In [0]:
Transformed_df.head()

In [0]:
#Correlation heatmap
# Select only numerical columns
numerical_cols = ['age_ordinal', 'quantity', 'total_revenue', 'quantity', 'cust_segment_ordinal']
corr_matrix = Transformed_df[numerical_cols].corr()

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f",
            cbar_kws={'label': 'Correlation Coefficient'})
plt.title('Correlation Matrix of Numerical Features')
plt.show()

In [0]:
plt.figure(figsize=(12, 8))
sns.countplot(x='cust_segment_ordinal', hue='age_ordinal', data=Transformed_df)
plt.title('Purchase Count by Age')
plt.xticks(rotation=45)
plt.show()

In [0]:
plt.figure(figsize=(12, 8))
sns.countplot(x='payment_method', hue= 'cust_segment_ordinal', data=Transformed_df)
plt.title('Purchase Count by Payment Method')
plt.xticks(rotation=45)
plt.show()

In [0]:
#Transformed_df['month'] = Transformed_df['invoice_date'].dt.month
Transformed_df['month_name'] = Transformed_df['invoice_date'].dt.strftime('%B')
Transformed_df.head()

In [0]:
Transformed_df['day_of_week'] = Transformed_df['invoice_date'].dt.day_name()
Transformed_df.head()

In [0]:
# Aggregate total quantity and price by month
monthly_sales = Transformed_df.groupby('month_name')[['quantity', 'price']].sum().reset_index()

# Plot monthly sales trends
plt.figure(figsize=(12, 6))
sns.lineplot(x='month_name', y='price', data=monthly_sales)
plt.title('Monthly Sales Trend (Price)')
plt.xlabel('Month')
plt.ylabel('Total Price')
plt.xticks(monthly_sales['month_name'], ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.show()

In [0]:
# Also, let's analyze by day of the week
day_of_week_sales = Transformed_df.groupby('day_of_week')['quantity'].sum().reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']).reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='day_of_week', y='quantity', data=day_of_week_sales)
plt.title('Sales Quantity by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Total Quantity Sold')
plt.show()

In [0]:
# Also, let's analyze by day of the week
day_of_week_sales = Transformed_df.groupby('day_of_week')['price'].sum().reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']).reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='day_of_week', y='price', data=day_of_week_sales)
plt.title('Sales by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Total Sales')
plt.show()

In [0]:
%pip install databricks-feature-engineering

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS retail_catalog;
CREATE SCHEMA IF NOT EXISTS retail_catalog.retail_transformed;

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient
import pyspark.sql.functions as F

In [0]:
spark1_df = spark.createDataFrame(Transformed_df)

# 2. Engineer features from the raw data

# 3. Initialize the Feature Engineering client
fe = FeatureEngineeringClient()

In [0]:
feature_table_name = "retail_catalog.retail_transformed"
primary_keys = ["invoice_no"]

# 5. Create the feature table and write the data

# The `create_table` method takes the DataFrame to infer the schema.
# `write_table` populates the table with data.
fe.create_table(
    name=feature_table_name,
    df=spark1_df,
    primary_keys = primary_keys
    #mode="merge" # or 'merge' for incremental updates
)